In [1]:
import numpy as np

http://www.math.unl.edu/~s-jeverso2/McElieceProject.pdf

In [2]:
m = np.array([1, 1, 0, 1])

In [3]:
G = np.array([
    [1, 1, 0, 1, 0, 0, 1],
    [1, 1, 1, 0, 0, 0, 0],
    [1, 0, 0, 1, 1, 0, 0],
    [0, 1, 0, 1, 0, 1, 0]])

P = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0 ,1],
    [1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 0]])

S = np.array([
    [1, 1, 0, 1],
    [1, 0, 0, 1],
    [0, 1, 1, 1], 
    [1, 1, 0, 0]])

In [4]:
G_ = S.dot(G)%2
G_ = G_.dot(P)%2
G_

array([[0, 0, 0, 1, 1, 1, 1],
       [0, 1, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 1, 1],
       [1, 0, 0, 0, 1, 0, 1]])

In [5]:
c = m.dot(G_)%2
c

array([1, 1, 0, 1, 1, 0, 0])

In [6]:
ei = np.random.randint(c.shape[0])
e = np.zeros(c.shape[0])
e[ei] = 1
e

array([0., 0., 0., 0., 1., 0., 0.])

In [7]:
c = (c + e)%2
c

array([1., 1., 0., 1., 0., 0., 0.])

In [8]:
P1 = np.linalg.inv(P)%2
S1 = np.linalg.inv(S)%2 

In [9]:
P1

array([[0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0.]])

In [10]:
c_m = c.dot(P1)
print(c_m)
c_m = c_m[[2,4,5,6]]
print(c_m)

[1. 1. 0. 1. 0. 0. 0.]
[0. 0. 0. 0.]


In [11]:
S1

array([[1., 1., 0., 1.],
       [1., 1., 0., 0.],
       [0., 1., 1., 1.],
       [1., 0., 0., 1.]])

In [12]:
m_ = c_m.dot(S1)%2
m_

array([0., 0., 0., 0.])